In [1]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
import ipympl

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

In [2]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_derived = '/Users/alina/Desktop/MIT/code/data/output/derived_data'

In [3]:
baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
baseline_var_short = ['src_subject_id', 'days_baseline']

dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}

version_form = ['version_form']

qsts = ['snap', 'ssrs',  'masc', 'pc']##, 'wechsler'] #masc to many missing data 

In [4]:
interaction_predictors = ['days_baseline', 'site', 'trtname'] #time, site, treatment group

# mediator variables
comorb_mediators  = ['cdorodd' , 'pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa'] #ODD/CD or anx excluding specific phobia 
services_mediators =  ['demo61'] #reciept of public assistance 
prev_med_mediators = ['hi_24'] #medication intake prior to study 

#moderator variables 
accept_moderator = ['d2dresp'] # initail acceptance of treatment 
raters = ['Teacher', 'Parent']

med_mod_list = np.concatenate([[comorb_mediators[0]], ['anx'], services_mediators, prev_med_mediators, accept_moderator])
med_mod_list

array(['cdorodd', 'anx', 'demo61', 'hi_24', 'd2dresp'], dtype='<U7')

In [5]:
treat_group_file = 'treatment_groups.csv'
treat_group = pd.read_csv(Path(data_derived, treat_group_file))

In [6]:
diags1_file  = 'diagpsx01.txt' #comorbid anx and cd/odd 
demog_file = 'demgr01.txt' ##public assistance 
health_qst_file  = 'health01.txt' #prev medication 
initial_sat_file = 'debrief01.txt' #initial acceptance of treatment arm 

diags1 = pd.read_csv(Path(data_root, diags1_file), delimiter = '\t', skiprows=[1])
demog = pd.read_csv(Path(data_root, demog_file), delimiter= '\t', skiprows=[1])
health_qst = pd.read_csv(Path(data_root, health_qst_file), delimiter='\t', skiprows=[1])
init_sat = pd.read_csv(Path(data_root, initial_sat_file), delimiter='\t', skiprows=[1])

med_qsts = ['diags', 'demog', 'heath_qst', 'init_sat']
med_qsts_dict = dict(zip(med_qsts, [diags1, demog, health_qst, init_sat]))


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_24216/885613445.py:6: DtypeWarning: Columns (9,16,63,64,120,122,125,126,127,128,129,130,138,140,142,144,147,150,152,159,170,172,173,174,175,180,183,184,185,186,187,188,190,192,197,198,960) have mixed types. Specify dtype option on import or set low_memory=False.
  diags1 = pd.read_csv(Path(data_root, diags1_file), delimiter = '\t', skiprows=[1])
/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_24216/885613445.py:8: DtypeWarning: Columns (18,20,23,26,29,32,35,42,50,52,54,59,69,76,85,87,368,374,380,386,392,398) have mixed types. Specify dtype option on import or set low_memory=False.
  health_qst = pd.read_csv(Path(data_root, health_qst_file), delimiter='\t', skiprows=[1])


In [7]:
diags1 = pd.read_csv(Path(data_root, diags1_file), delimiter = '\t')
demog = pd.read_csv(Path(data_root, demog_file), delimiter= '\t')
health_qst = pd.read_csv(Path(data_root, health_qst_file), delimiter='\t')
init_sat = pd.read_csv(Path(data_root, initial_sat_file), delimiter='\t')

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_24216/1318834801.py:1: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,2

In [8]:
odd_cd_vars =[ 'cdorodd'] # CD or ODD 
anx_vars = ['pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa', 'psai'] #poa : overanxious disorder: see if included in alaysis
comorb_vars = np.concatenate([odd_cd_vars, anx_vars])

assist_vars = ['demo61']# public assistance 

prev_med_vars = ['hi_24'] #prev medication 

accept_vars = ['d2dresp']# initial acceptance 

In [9]:
for qst in med_qsts_dict.values():
    print(version_form[0] in qst.keys())

True
False
False
False


In [10]:
# comorb = pd.read_csv(Path(data_derived, 'comorb.csv'), index_col= 0) # comorbid ODD/CD anxiety
# prev_med = pd.read_csv(Path(data_derived, 'prev_med.csv'), index_col= 0) #'prevous stimulant medictaion exposure
# assist = pd.read_csv(Path(data_derived, 'assist.csv'), index_col= 0) #reciept of public assistance 
# accept = pd.read_csv(Path(data_derived, 'accept.csv'), index_col= 0) #initial acceptance of treatment arm
# medi = {'comorb': comorb, 'prev_med': prev_med, 'assist': assist, 'accept': accept}

# comorbidities

In [11]:
comorb = diags1[np.concatenate([baseline_var_short,version_form, comorb_vars])]
assist = demog[np.concatenate([baseline_var_short, assist_vars])]
prev_med = health_qst[np.concatenate([baseline_var_short, prev_med_vars])]
init_acc = init_sat[np.concatenate([baseline_var_short, accept_vars])]

In [12]:
comorb['anx'] = pd.NA
anx_vars = [ 'pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai', 'psa', 'psai']
mask = (comorb[anx_vars].fillna(0) != 0).any(axis=1)  # Extract if any anxiety disdoers, comorb anx = True 
comorb.loc[mask, 'anx'] = 1
mask = (comorb[anx_vars] == 0).all(axis=1)  # If none of the disorder, Comorb anx = False, else NaN 
comorb.loc[mask, 'anx'] = 0
comorb = comorb.drop(columns=anx_vars)

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_24216/3060119594.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comorb['anx'] = pd.NA


## split data from timepoints so that only 1 diagnosis per timepoint per subject

In [13]:
timepoints = [50, 213, 578, 912] 

In [14]:
def find_unique_subjects(df, lower_bound_time=None, select='min'):
    if lower_bound_time is not None:
        df = df.loc[df['days_baseline'] > lower_bound_time].copy()
    
    # Fix for the error - Using value_counts and filtering properly
    duplicates = df['src_subject_id'].value_counts()[df['src_subject_id'].value_counts() > 1].index
    duplicates_df = df[df['src_subject_id'].isin(duplicates)]

    if select == 'min':
        idx_to_keep = duplicates_df.groupby('src_subject_id')['days_baseline'].idxmin()
    elif select == 'max':
        idx_to_keep = duplicates_df.groupby('src_subject_id')['days_baseline'].idxmax()
    else:
        raise ValueError('Please specify how to select unique subjects')
    
    return df.loc[df.index.difference(duplicates_df.index).union(idx_to_keep)]

def get_unique_subjects_split(df, timepoints_unique = None, select='min', timepoints_split = None):
    df_split = prep.split_data_from_timepoints(df, timepoints_split)
    
    if timepoints_unique is not None:
        df_split_unique = [
            find_unique_subjects(df_timepoint, time, select) for df_timepoint, time in zip(df_split.values(), timepoints_unique)
        ]
    else:
        df_split_unique = [
            find_unique_subjects(df_timepoint, None, select) for df_timepoint in df_split.values()
        ]
    
    for df_ in df_split_unique:
        if (df_['src_subject_id'].value_counts() > 1).sum() == 0:
            print('Success')
        else:
            print('Found {} duplicates remaining.'.format((df_['src_subject_id'].value_counts() > 1).sum()))
    
    return df_split_unique


In [15]:
timepoints

[50, 213, 578, 912]

In [16]:
comorb_split = prep.split_data_from_timepoints(comorb)
comorb_split_unique = [ find_unique_subjects(comorb_timepoint, None, select='min') for comorb_timepoint, time in zip(comorb_split.values(), timepoints) ]
for df in comorb_split_unique:
    print((df['src_subject_id'].value_counts() > 1).sum())

0
0
0
0


In [21]:
comorb_split_unique[0]["days_baseline"]

6664    0
6702    0
6721    0
6747    0
6753    0
       ..
9324    0
9326    0
9328    0
9330    0
9332    0
Name: days_baseline, Length: 273, dtype: int64

In [19]:
comorb_split_unique = get_unique_subjects_split(comorb, None, select='min', timepoints_split=[213, 578, 912, 1095])

Success
Success
Success
Success


In [20]:
comorb_split_unique[0].shape

(867, 5)

## Previous medication 

In [21]:
prev_med_split_unique = get_unique_subjects_split(prev_med,  None, select='min', timepoints_split=[213, 578, 912, 1095])

Success
Success
Success
Success


In [22]:
prev_med

,src_subject_id,days_baseline,hi_24
0,Subject ID how it's defined in lab/project,Days since baseline,Have you ever taken ADHD medication?
1,P1001,3029,NaN
2,P1001,3836,1
3,P1001,4786,1
4,P1001,5297,1
...,...,...,...
2823,P1867,4919,1.0
2824,P1868,3114,NaN
2825,P1868,3828,1.0
2826,P1868,4627,1.0


In [23]:
prev_med['days_baseline'].value_counts()

days_baseline
3734    7
3711    6
3731    5
3821    5
4440    5
       ..
4452    1
5040    1
2769    1
5905    1
5251    1
Name: count, Length: 2165, dtype: int64

In [24]:
prev_med_unique = find_unique_subjects(prev_med)
prev_med_unique

TypeError: '<' not supported between instances of 'int' and 'str'

### recipet of public assistance 

In [25]:
assist_unique = find_unique_subjects(assist)
assist_unique

,src_subject_id,days_baseline,demo61
0,Subject ID how it's defined in lab/project,Days since baseline,Does any part of your annual household income ...
1,P1001,1612,2
4,P1003,2647,2
7,P1004,2633,2
10,P1006,2560,2
...,...,...,...
3856,P1780,4108,NaN
3913,P1804,3901,NaN
3947,P1818,4530,NaN
3964,P1826,4858,NaN


In [26]:
assist_unique['demo61'].unique()

array(['Does any part of your annual household income come from public assistance?',
       '2', '1', nan], dtype=object)

In [27]:
init_acc

,src_subject_id,days_baseline,d2dresp
0,Subject ID how it's defined in lab/project,Days since baseline,Which of the following choices best describes ...
1,P1002,0,NaN
2,P1003,0,NaN
3,P1004,0,NaN
4,P1005,0,NaN
...,...,...,...
575,P1830,0,6
576,P1837,0,4
577,P1839,0,5
578,P1842,0,3


In [28]:
# init_acc['d2dresp_bin'] = pd.NA
# init_acc.loc[init_acc['d2dresp']] = init_acc.loc[init_acc['d2dresp']].astype(int)
# init_acc.loc[init_acc['d2dresp'] <= 2, 'd2dresp_bin'] = 0
# init_acc.loc[init_acc['d2dresp'] > 2, 'd2dresp_bin'] = 1

### Initial acceptance is already unique

In [29]:
comorb_b = comorb_split_unique[0]

drop days baseline and version form for merging 

In [31]:
comorb_b = comorb_b.drop(columns='version_form')

In [32]:
dfs_clean = [df.drop(columns='days_baseline') for df in [comorb_b, prev_med_unique, assist_unique, init_acc]]

NameError: name 'prev_med_unique' is not defined

In [33]:
dfs_clean

NameError: name 'dfs_clean' is not defined

## Merge with data for the regression 

In [202]:
baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}

version_form = ['version_form']

qsts = ['snap', 'ssrs',  'masc', 'pc']##, 'wechsler'] #masc to many missing data 

In [1]:
snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
masc_file = 'masc_p01.txt'
parent_child_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'
treat_group_file = 'treatment_groups.csv'
#outcome variablles 
snap_vars = ['snainatx', 'snahix', 'snaoddx'] #inattention_mean, hyperactie mean
ssrs_vars = ['sspintx', 'ssptossx']# social skills mean, internalizing mean 
masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
#wechsler_vars = ['w1readb','w2math','w3spell' ]
outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars, 'masc':masc_vars ,'pc': pc_vars} #, 'wechsler': wechsler_vars}

In [2]:
treat_group = pd.read_csv(Path(data_derived, treat_group_file))

snap = prep.get_data(Path(data_root, snap_file), columns= [baseline_var, snap_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
ssrs = prep.get_data(Path(data_root, ssrs_file), columns= [baseline_var, ssrs_vars, version_form], treat_group= treat_group, set_dtypes= True, version_form= True, split_timepoints= True)
masc = prep.get_data(Path(data_root, masc_file), columns= [baseline_var, masc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= True)

pc = prep.get_data(Path(data_root, parent_child_file), columns= [baseline_var, pc_vars], treat_group= treat_group, set_dtypes= True, version_form= False, split_timepoints= True)
#wechsler = prep.get_data(Path(data_root, wechsler_file), columns= [baseline_var, wechsler_vars], set_dtypes= True, version_form= False, split_timepoints= True)
data_dict = dict(zip(qsts, [snap, ssrs, masc, pc]))

NameError: name 'pd' is not defined

In [205]:
data_14_months = [data['14'] for data in data_dict.values()]

In [206]:
test = [pd.merge(data, dfs_clean[0], on= 'src_subject_id') for data in data_14_months]
test2 = [pd.merge(data, dfs_clean[1], on= 'src_subject_id') for data in test]
test3 = [pd.merge(data, dfs_clean[2], on= 'src_subject_id') for data in test2]
test4 = [pd.merge(data, dfs_clean[3], on= 'src_subject_id') for data in test3]


In [207]:
# test4[0].to_csv(Path(data_derived, 'snap_14_and_mediators.csv'))
# test4[1].to_csv(Path(data_derived, 'ssrs_14_and_mediators.csv'))
# test4[2].to_csv(Path(data_derived, 'masc_14_and_mediators.csv'))
# test4[3].to_csv(Path(data_derived, 'pc_14_and_mediators.csv'))

In [143]:
len(test4)

4